In [5]:
# coding: utf-8

In [1]:
import os
import sys
from pathlib import Path
import pandas as pd
import sqlite3
from sqlalchemy import create_engine
from sqlalchemy.sql import select

In [2]:
sys.path.append(r"../src/collectcube")
from db import *

### Check that pixel table already exists

In [3]:
sql_db_path = 'sqlite:///../data/db_samp/landcover.db'
engine = create_engine(sql_db_path, echo=False)
with engine.connect() as conn:
    df = pd.read_sql_table('pixels', conn)
df.head()

,PID,Center,cent_lat,cent_long,cent_X,cent_Y,ransamp
0,0000001_0,1,-26.626629,-57.702188,2937350.0,-3358400.0,1
1,0000002_0,1,-25.953394,-56.601457,3045480.0,-3276600.0,1
2,0000003_0,1,-25.328468,-56.187909,3090550.0,-3200450.0,1
3,0000004_0,1,-26.452609,-55.682417,3123190.0,-3337280.0,1
4,0000005_0,1,-25.633537,-55.052163,3190670.0,-3237650.0,1


## Make sql lc table from existing lut.csv

In [ ]:
local_db_path = '../data/db_samp/landcover.db'
lc_lut = Path("C:/Users/klobw/Desktop/NasaProject/RemoteComputing/LUCinSA_helpers/Class_LUT.csv")
make_LC_table_from_lut(lc_lut, local_db_path, treat_existing='fail')

### check if it worked:

In [13]:
with engine.connect() as con:
    df = pd.read_sql_table('LC', con)
df.head()

,LC_UNQ,USE_NAME,LC5,LC25
0,1,NoVeg,10,99
1,2,NoVeg_Bare,10,2
2,3,NoVeg_Built,10,3
3,7,NoVeg_Water,10,7
4,9,Mixed-path,10,9


### Add other tables to database using sqlalchemy to control foreign keys

In [9]:
def make_db(sql_db_path):
    engine = create_engine(sql_db_path, echo=False)
    make_main_support_tables(engine)
    populate_LC5_table(engine)
    
make_db(sql_db_path)

### Check that tables are made and populated

In [7]:
with engine.connect() as conn:
    lc5_table = Table('LC5', MetaData(), autoload_with=engine)
    rp = conn.execute(lc5_table.select())
results = rp.fetchall() 
print(results)

[(10, 'NoVeg'), (20, 'LowVeg'), (40, 'MedVeg'), (50, 'HighVeg'), (70, 'Trees'), (99, 'Unknown')]


In [6]:
with engine.connect() as con:
    df = pd.read_sql_table('LC5', con)
df.head()

,LC5id,LC5type
0,10,NoVeg
1,20,LowVeg
2,40,MedVeg
3,50,HighVeg
4,70,Trees
